In [1]:
import cv2
import numpy as np
from glob import glob
# np.set_printoptions(suppress=True, formatter={'float_kind':'{:f}'.format})

from config import *
from utils import *

In [2]:
def build_k_matrix(kkt,resize_f=8):
    k = np.zeros((3,3))
    k[:,-1] = kkt[:,-1]
    k[1,1] = np.sqrt(kkt[1,1]-kkt[1,2]**2)
    k[0,1] = (kkt[0,1] - kkt[0,2]*kkt[1,2])/k[1,1]
    k[0,0] = np.sqrt(kkt[0,0]-k[0,1]**2-kkt[0,2]**2)
    k = k*resize_f;k[-1,-1] = 1
    return k

In [3]:
files = sorted(glob('*.jpg'))
COORDS = np.asarray(COORDS,dtype=np.int32)

In [4]:
# Checkerboard pattern, vanishing points, full K matrix

COORDS = [[[353,233],[406,248],[341,255],[396,270]],\
          [[348,263],[402,249],[372,288],[426,273]],\
          [[277,275],[337,276],[271,294],[333,295]],\
          [[405,278],[467,279],[412,316],[472,316]],\
          [[373,154],[421,163],[340,212],[388,222]]]

A = np.zeros((5,6))
for k,file in enumerate(files):
    if k < 5:
        coords = COORDS[k]
        p1,p2,l_inf = find_l_infinity(coords)
        print(p1,p2,l_inf)
        A[k] = np.asarray([p1[0]*p2[0],p1[1]*p2[1],1,p1[0]*p2[1]+p1[1]*p2[0],p1[0]+p2[0],p1[1]+p2[1]])

_,_,vh = np.linalg.svd(A)
x = vh.T[:,-1]
w = np.asarray([[x[0],x[3],x[4]],[x[3],x[1],x[5]],[x[4],x[5],x[2]]])
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=8)
print('k:')
print(k)
print('w:')
w = np.linalg.inv(np.matmul(k,k.T))
print(np.divide(w,w[-1,-1]))

[2.80866667e+03 9.28000000e+02 1.00000000e+00] [ 711.90909091 -425.            1.        ] [-7.29641635e-04  1.13073291e-03  1.00000000e+00]
[ 2.05800000e+03 -1.80333333e+02  1.00000000e+00] [-1.284e+03 -1.437e+03  1.000e+00] [-3.94076023e-04  1.04801226e-03  1.00000000e+00]
[3.5797e+04 8.6700e+02 1.0000e+00] [ 457.63157895 -297.            1.        ] [-1.05544959e-04  3.20437473e-03  1.00000000e+00]
[2.761e+03 3.160e+02 1.000e+00] [6.37217391e+02 1.53860870e+03 1.00000000e+00] [-3.02122036e-04 -5.24813480e-04  1.00000000e+00]
[-2.74100e+03 -4.29875e+02  1.00000e+00] [ 3.50200000e+03 -5.34545455e+03  1.00000000e+00] [3.04232486e-04 3.86388500e-04 1.00000000e+00]
k:
[[3.15201387e+03 1.27730573e+02 3.92106852e+03]
 [0.00000000e+00 2.25620118e+03 3.91588240e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
w:
[[ 1.86745753e-08 -1.05722585e-09 -6.90843171e-05]
 [-1.05722585e-09  3.65076538e-08 -1.38814224e-04]
 [-6.90843171e-05 -1.38814224e-04  1.00000000e+00]]


In [5]:
# Checkerboard pattern, vanishing points, zero skew condition

from itertools import combinations 
comb = list(combinations(np.arange(5), 3))
# file_list = np.random.choice(5,3,replace=False)

COORDS_1 = [[[353,233],[406,248],[341,255],[396,270]],\
          [[348,263],[402,249],[372,288],[426,273]],\
          [[277,275],[337,276],[271,294],[333,295]],\
          [[405,278],[467,279],[412,316],[472,316]],\
          [[373,154],[421,163],[340,212],[388,222]]]

comb = [[0,3,4]]
for lists in comb:
    A = np.zeros((3,4))
    i = 0
    print('*'*50, lists, '*'*50)
    for k,file in enumerate(files):
        if k in lists:
            coords = COORDS_1[k]
            p1,p2,l_inf = find_l_infinity(coords)
            print(p1,p2,l_inf)
            A[i] = np.asarray([p1[0]*p2[0]+p1[1]*p2[1],1,p1[0]+p2[0],p1[1]+p2[1]])
            i += 1

    _,_,vh = np.linalg.svd(A)
    x = vh.T[:,-1]
    w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
#     print('w:')
#     print(np.divide(w,w[-1,-1]))
    kkt = np.linalg.pinv(w)
    kkt = np.divide(kkt,kkt[-1,-1])
    # print(kkt)
    # ev,_ = np.linalg.eig(kkt)
    # # k = np.linalg.cholesky(kkt)
    k = build_k_matrix(kkt)
    print('k:')
    print(k)
    print('w:')
    w = np.linalg.inv(np.matmul(k,k.T))
    print(np.divide(w,w[-1,-1]))


************************************************** [0, 3, 4] **************************************************
[2.80866667e+03 9.28000000e+02 1.00000000e+00] [ 711.90909091 -425.            1.        ] [-7.29641635e-04  1.13073291e-03  1.00000000e+00]
[2.761e+03 3.160e+02 1.000e+00] [6.37217391e+02 1.53860870e+03 1.00000000e+00] [-3.02122036e-04 -5.24813480e-04  1.00000000e+00]
[-2.74100e+03 -4.29875e+02  1.00000e+00] [ 3.50200000e+03 -5.34545455e+03  1.00000000e+00] [3.04232486e-04 3.86388500e-04 1.00000000e+00]
k:
[[9.49628678e+03 8.00269872e-10 1.42582258e+04]
 [0.00000000e+00 9.49628678e+03 5.08146410e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
w:
[[ 3.13187306e-09 -2.63903832e-22 -4.46549533e-05]
 [-2.63820713e-22  3.13187306e-09 -1.59145005e-05]
 [-4.46549533e-05 -1.59145005e-05  1.00000000e+00]]


In [6]:
# Checkerboard pattern, homography, full K matrix

coords_checker_board = [[[353,233],[406,248],[341,255],[396,270]],\
                        [[348,263],[402,249],[372,288],[426,273]],\
                        [[373,154],[421,163],[340,212],[388,222]]]

actual_coords_checker_board = [[[350,230],[405,230],[350,285],[405,285]],\
                               [[350,230],[405,230],[350,285],[405,285]],\
                               [[350,230],[405,230],[350,285],[405,285]]]

X = np.zeros((6,6))
for i in range(len(coords_checker_board)):
    A = np.zeros((8,9))
    coords = coords_checker_board[i]
    actual_coords = actual_coords_checker_board[i]
    for j in range(len(coords)):
        x_,y_ = coords[j][0],coords[j][1]
        x,y = actual_coords[j][0],actual_coords[j][1]
        A[2*j,:3] = [-x,-y,-1]
        A[2*j,-3:] = [x*x_,y*x_,x_]
        A[2*j+1,3:6] = [-x,-y,-1]
        A[2*j+1,-3:] = [x*y_,y*y_,y_]

    _,_,vh = np.linalg.svd(A)
    h = vh.T[:,-1].reshape(3,3)
    h = np.divide(h,h[-1,-1])
    print('h:')
    print(h)
    h1 = h[:,0];h2 = h[:,1]
    X[2*i] = np.asarray([h1[0]*h2[0], h1[1]*h2[1], h1[2]*h2[2], h1[0]*h2[1]+h1[1]*h2[0],
                        h1[0]*h2[2]+h1[2]*h2[0], h1[1]*h2[2]+h1[2]*h2[1]])
    t2 = np.asarray([h2[0]*h2[0], h2[1]*h2[1], h2[2]*h2[2], h2[0]*h2[1]+h2[1]*h2[0],
                        h2[0]*h2[2]+h2[2]*h2[0], h2[1]*h2[2]+h2[2]*h2[1]])
    t1 = np.asarray([h1[0]*h1[0], h1[1]*h1[1], h1[2]*h1[2], h1[0]*h1[1]+h1[1]*h1[0],
                        h1[0]*h1[2]+h1[2]*h1[0], h1[1]*h1[2]+h1[2]*h1[1]])
    X[2*i+1] = t1-t2

_,_,vh = np.linalg.svd(X)
x = vh.T[:,-1]
w = np.asarray([[x[0],x[3],x[4]],[x[3],x[1],x[5]],[x[4],x[5],x[2]]])
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
# k = np.linalg.cholesky(kkt)
k = build_k_matrix(kkt,resize_f=8)
print('k:')
print(k)
print('w:')
w = np.linalg.pinv(np.matmul(k,k.T))
print(np.divide(w,w[-1,-1]))

h:
[[ 1.13222252e+00 -4.20908358e-01  5.53335125e+01]
 [ 3.74092986e-01  2.51276539e-01  4.54635851e+01]
 [ 4.03117442e-04 -5.91238914e-04  1.00000000e+00]]
h:
[[ 1.43027027e+00  3.96602317e-01 -1.83887259e+02]
 [-1.25328185e-01  4.43861004e-01  2.50065637e+02]
 [ 6.94980695e-04 -3.08880309e-04  1.00000000e+00]]
h:
[[ 6.63479576e-01 -5.82783661e-01  2.28945083e+02]
 [ 1.04054463e-01  8.89561271e-01 -1.05959455e+02]
 [-2.42057489e-04 -1.66414523e-04  1.00000000e+00]]
k:
[[ 9.57524207e+03 -1.42633384e+03  9.21023831e+03]
 [ 0.00000000e+00  9.19046456e+03  2.31057174e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
w:
[[ 5.28980321e-09  8.20962343e-10 -5.06172406e-05]
 [ 8.20962343e-10  5.86942320e-09 -2.11229822e-05]
 [-5.06172406e-05 -2.11229822e-05  1.00000000e+00]]


In [7]:
# Checkerboard pattern, homography, zero skew matrix

coords_checker_board = [[[353,233],[406,248],[341,255],[396,270]],\
                        [[348,263],[402,249],[372,288],[426,273]]]

actual_coords_checker_board = [[[350,230],[405,230],[350,285],[405,285]],\
                               [[350,230],[405,230],[350,285],[405,285]]]

X = np.zeros((4,4))
for i in range(len(coords_checker_board)):
    A = np.zeros((8,9))
    coords = coords_checker_board[i]
    actual_coords = actual_coords_checker_board[i]
    for j in range(len(coords)):
        x_,y_ = coords[j][0],coords[j][1]
        x,y = actual_coords[j][0],actual_coords[j][1]
        A[2*j,:3] = [-x,-y,-1]
        A[2*j,-3:] = [x*x_,y*x_,x_]
        A[2*j+1,3:6] = [-x,-y,-1]
        A[2*j+1,-3:] = [x*y_,y*y_,y_]

    _,_,vh = np.linalg.svd(A)
    h = vh.T[:,-1].reshape(3,3)
    h = np.divide(h,h[-1,-1])
    print('h:')
    print(h)
    h1 = h[:,0];h2 = h[:,1]
    X[2*i] = np.asarray([h1[0]*h2[0] + h1[1]*h2[1], h1[2]*h2[2], h1[0]*h2[2]+h1[2]*h2[0], h1[1]*h2[2]+h1[2]*h2[1]])
    t2 = np.asarray([h2[0]*h2[0] + h2[1]*h2[1], h2[2]*h2[2], h2[0]*h2[2]+h2[2]*h2[0], h2[1]*h2[2]+h2[2]*h2[1]])
    t1 = np.asarray([h1[0]*h1[0] + h1[1]*h1[1], h1[2]*h1[2], h1[0]*h1[2]+h1[2]*h1[0], h1[1]*h1[2]+h1[2]*h1[1]])
    X[2*i+1] = t1-t2

_,_,vh = np.linalg.svd(X)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
# print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=8)
print('k:')
print(k)
print('w:')
w = np.linalg.pinv(np.matmul(k,k.T))
print(np.divide(w,w[-1,-1]))

h:
[[ 1.13222252e+00 -4.20908358e-01  5.53335125e+01]
 [ 3.74092986e-01  2.51276539e-01  4.54635851e+01]
 [ 4.03117442e-04 -5.91238914e-04  1.00000000e+00]]
h:
[[ 1.43027027e+00  3.96602317e-01 -1.83887259e+02]
 [-1.25328185e-01  4.43861004e-01  2.50065637e+02]
 [ 6.94980695e-04 -3.08880309e-04  1.00000000e+00]]
[[ 3.33252538e-07 -0.00000000e+00 -4.25504915e-04]
 [-0.00000000e+00  3.33252538e-07 -5.74050759e-05]
 [-4.25504915e-04 -5.74050759e-05  1.00000000e+00]]
k:
[[9.26334628e+03 3.62541689e-09 1.02145938e+04]
 [0.00000000e+00 9.26334628e+03 1.37805584e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
w:
[[ 5.20707091e-09 -2.03799202e-21 -5.31881144e-05]
 [-2.04343314e-21  5.20707091e-09 -7.17563448e-06]
 [-5.31881144e-05 -7.17563448e-06  1.00000000e+00]]


In [15]:
def get_K_from_w(w):
    w_inv = np.linalg.pinv(w)
    w_inv = w_inv/w_inv[2,2]
#     print('Inverse of w is :',w_inv)
    ux = w_inv[0,2]
    uy = w_inv[1,2]
    fy = np.sqrt(w_inv[1,1] - uy**2)
    s = (w_inv[0,1] - ux*uy)/fy
    fx = np.sqrt(w_inv[0,0] - s*2 - ux*2)
    K = np.array([[fx,s,ux],[0,fy,uy],[0,0,1]])
    return K

In [17]:
# sumanth points
# Checkerboard pattern, homography, zero skew matrix

# np.array([[,0,10],[3304,1792,10,10],[,],[,]])
# points12 = np.array([[,0,10],[,10,10],[,0,0],[,10,0]])
coords_checker_board = [[[2496,1560],[3304,1792],[2304,1904],[3160,2152]],\
                        [[2742,2209],[3236,2216],[2779,2509],[3290,2515]]]

actual_coords_checker_board = [[[0,10],[10,10],[0,0],[10,0]],\
                               [[0,10],[10,10],[0,0],[10,0]]]

X = np.zeros((4,4))
for i in range(len(coords_checker_board)):
    A = np.zeros((8,9))
    coords = coords_checker_board[i]
    actual_coords = actual_coords_checker_board[i]
    for j in range(len(coords)):
        x_,y_ = coords[j][0],coords[j][1]
        x,y = actual_coords[j][0],actual_coords[j][1]
        A[2*j,:3] = [-x,-y,-1]
        A[2*j,-3:] = [x*x_,y*x_,x_]
        A[2*j+1,3:6] = [-x,-y,-1]
        A[2*j+1,-3:] = [x*y_,y*y_,y_]

    _,_,vh = np.linalg.svd(A)
    h = vh.T[:,-1].reshape(3,3)
    h = np.divide(h,h[-1,-1])
    print('h:')
    print(h)
    h1 = h[:,0];h2 = h[:,1]
    X[2*i] = np.asarray([h1[0]*h2[0] + h1[1]*h2[1], h1[2]*h2[2], h1[0]*h2[2]+h1[2]*h2[0], h1[1]*h2[2]+h1[2]*h2[1]])
    t2 = np.asarray([h2[0]*h2[0] + h2[1]*h2[1], h2[2]*h2[2], h2[0]*h2[2]+h2[2]*h2[0], h2[1]*h2[2]+h2[2]*h2[1]])
    t1 = np.asarray([h1[0]*h1[0] + h1[1]*h1[1], h1[2]*h1[2], h1[0]*h1[2]+h1[2]*h1[0], h1[1]*h1[2]+h1[2]*h1[1]])
    X[2*i+1] = t1-t2

_,_,vh = np.linalg.svd(X)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
print('k_1:')
print(get_K_from_w(w))
# print('w:')
# print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = np.linalg.cholesky(kkt)
# k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)
print('w:')
w = np.linalg.pinv(np.matmul(k,k.T))
print(np.divide(w,w[-1,-1]))

# import scipy




h:
[[ 8.38537991e+01  3.42735346e+01  2.30400000e+03]
 [ 2.36108151e+01 -2.49790409e+01  1.90400000e+03]
 [-5.52595224e-04  6.03907638e-03  1.00000000e+00]]
h:
[[ 5.24688980e+01  5.86074473e+00  2.77900000e+03]
 [ 1.64643720e+00 -2.22977078e+01  2.50900000e+03]
 [ 4.16078410e-04  3.48677780e-03  1.00000000e+00]]
k_1:
[[ 1.52257040e+04 -1.54337340e-07  4.13412831e+03]
 [ 0.00000000e+00  1.46539863e+04 -4.10445764e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
k:
[[ 1.52259756e+04  0.00000000e+00  0.00000000e+00]
 [-1.11443464e+03  1.51770855e+04  0.00000000e+00]
 [ 2.71518123e-01 -2.50500562e-01  9.29261738e-01]]
w:
[[ 4.02128216e-09  2.97855261e-22 -1.66244964e-05]
 [ 2.98850683e-22  4.02128216e-09  1.65051823e-05]
 [-1.66244964e-05  1.65051823e-05  1.00000000e+00]]


In [10]:
# Three squares, vanishing points, zero skew matrix

coords_3_squares = [[[405,356],[707,437],[254,578],[612,699]],\
          [[518,41],[830,154],[516,299],[767,428]],\
          [[54,107],[396,32],[125,380],[404,296]]]

A = np.zeros((3,4))
for k in range(A.shape[0]):
    coords = coords_3_squares[k]
    p1,p2,l_inf = find_l_infinity(coords)
    print(p1,p2,l_inf)
    A[k] = np.asarray([p1[0]*p2[0]+p1[1]*p2[1],1,p1[0]+p2[0],p1[1]+p2[1]])

_,_,vh = np.linalg.svd(A)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
# print(kkt)
# ev,_ = np.linalg.eig(kkt)
# k = np.linalg.cholesky(kkt)
k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)

[-3.50798966e+03 -6.93510472e+02  1.00000000e+00] [ 1.11470490e+03 -6.87407211e+02  1.00000000e+00] [-1.91656625e-06  1.45163388e-03  1.00000000e+00]
[-1.18877257e+03 -5.77158014e+02  1.00000000e+00] [5.06207437e+02 1.56224067e+03 1.00000000e+00] [ 0.00136704 -0.00108306  1.        ]
[ 3.65374394e+03 -6.82417532e+02  1.00000000e+00] [4.43676812e+02 1.60533478e+03 1.00000000e+00] [-3.70891328e-04 -5.20417378e-04  1.00000000e+00]
w:
[[ 5.86215934e-07 -0.00000000e+00 -2.52679370e-04]
 [-0.00000000e+00  5.86215934e-07 -2.95547073e-04]
 [-2.52679370e-04 -2.95547073e-04  1.00000000e+00]]
k:
[[ 1.12511649e+03 -2.54302340e-10  4.31034633e+02]
 [ 0.00000000e+00  1.12511649e+03  5.04160763e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
# Three squares, homography, full K matrix

coords_3_squares = [[[405,356],[707,437],[254,578],[612,699]],\
          [[518,41],[830,154],[516,299],[767,428]],\
          [[54,107],[396,32],[125,380],[404,296]]]

actual_coords_3_squares = [[[250,225],[625,225],[250,600],[625,600]],\
                 [[250,225],[625,225],[250,600],[625,600]],\
                 [[250,225],[625,225],[250,600],[625,600]]]
X = np.zeros((6,6))
for i in range(len(coords_3_squares)):
    A = np.zeros((8,9))
    coords = coords_3_squares[i]
    actual_coords = actual_coords_3_squares[i]
    for j in range(len(coords)):
        x_,y_ = coords[j][0],coords[j][1]
        x,y = actual_coords[j][0],actual_coords[j][1]
        A[2*j,:3] = [-x,-y,-1]
        A[2*j,-3:] = [x*x_,y*x_,x_]
        A[2*j+1,3:6] = [-x,-y,-1]
        A[2*j+1,-3:] = [x*y_,y*y_,y_]

    _,_,vh = np.linalg.svd(A)
    h = vh.T[:,-1].reshape(3,3)
    h = np.divide(h,h[-1,-1])
    print('h:')
    print(h)
    h1 = h[:,0];h2 = h[:,1]
    X[2*i] = np.asarray([h1[0]*h2[0], h1[1]*h2[1], h1[2]*h2[2], h1[0]*h2[1]+h1[1]*h2[0],
                        h1[0]*h2[2]+h1[2]*h2[0], h1[1]*h2[2]+h1[2]*h2[1]])
    t2 = np.asarray([h2[0]*h2[0], h2[1]*h2[1], h2[2]*h2[2], h2[0]*h2[1]+h2[1]*h2[0],
                        h2[0]*h2[2]+h2[2]*h2[0], h2[1]*h2[2]+h2[2]*h2[1]])
    t1 = np.asarray([h1[0]*h1[0], h1[1]*h1[1], h1[2]*h1[2], h1[0]*h1[1]+h1[1]*h1[0],
                        h1[0]*h1[2]+h1[2]*h1[0], h1[1]*h1[2]+h1[2]*h1[1]])
    X[2*i+1] = t1-t2

_,_,vh = np.linalg.svd(X[:5])
x = vh.T[:,-1]
w = np.asarray([[x[0],x[3],x[4]],[x[3],x[1],x[5]],[x[4],x[5],x[2]]])
print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)

h:
[[ 5.81296074e-01 -4.52284014e-01  3.07688764e+02]
 [ 1.14919072e-01  2.78910850e-01  2.17267392e+02]
 [-1.65706325e-04 -4.05743271e-04  1.00000000e+00]]
h:
[[ 4.99679031e-01  2.81181845e-01  3.40121099e+02]
 [ 2.42597923e-01  8.67774124e-01 -2.14082872e+02]
 [-4.20331898e-04  5.55467630e-04  1.00000000e+00]]
h:
[[ 1.28445859e+00  3.31014834e-01 -3.27782342e+02]
 [-2.39901064e-01  1.19769529e+00 -7.51406386e+01]
 [ 3.51545869e-04  7.46071972e-04  1.00000000e+00]]
w:
[[ 5.80017618e-07  6.59330542e-09 -2.57833110e-04]
 [ 6.59330542e-09  6.34064742e-07 -2.44722194e-04]
 [-2.57833110e-04 -2.44722194e-04  1.00000000e+00]]
k:
[[ 1.16939996e+03 -1.27146539e+01  4.40191030e+02]
 [ 0.00000000e+00  1.11851686e+03  3.81380424e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [12]:
# Three squares, homography, zero skew

coords_3_squares = [[[405,356],[707,437],[254,578],[612,699]],\
          [[518,41],[830,154],[516,299],[767,428]]]

actual_coords_3_squares = [[[250,225],[625,225],[250,600],[625,600]],\
                 [[250,225],[625,225],[250,600],[625,600]]]

X = np.zeros((4,4))
for i in range(len(coords_3_squares)):
    A = np.zeros((8,9))
    coords = coords_3_squares[i]
    actual_coords = actual_coords_3_squares[i]
    for j in range(len(coords)):
        x_,y_ = coords[j][0],coords[j][1]
        x,y = actual_coords[j][0],actual_coords[j][1]
        A[2*j,:3] = [-x,-y,-1]
        A[2*j,-3:] = [x*x_,y*x_,x_]
        A[2*j+1,3:6] = [-x,-y,-1]
        A[2*j+1,-3:] = [x*y_,y*y_,y_]

    _,_,vh = np.linalg.svd(A)
    h = vh.T[:,-1].reshape(3,3)
    h = np.divide(h,h[-1,-1])
    print('h:')
    print(h)
    h1 = h[:,0];h2 = h[:,1]
    X[2*i] = np.asarray([h1[0]*h2[0] + h1[1]*h2[1], h1[2]*h2[2], h1[0]*h2[2]+h1[2]*h2[0], h1[1]*h2[2]+h1[2]*h2[1]])
    t2 = np.asarray([h2[0]*h2[0] + h2[1]*h2[1], h2[2]*h2[2], h2[0]*h2[2]+h2[2]*h2[0], h2[1]*h2[2]+h2[2]*h2[1]])
    t1 = np.asarray([h1[0]*h1[0] + h1[1]*h1[1], h1[2]*h1[2], h1[0]*h1[2]+h1[2]*h1[0], h1[1]*h1[2]+h1[2]*h1[1]])
    X[2*i+1] = t1-t2

_,_,vh = np.linalg.svd(X)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)

h:
[[ 5.81296074e-01 -4.52284014e-01  3.07688764e+02]
 [ 1.14919072e-01  2.78910850e-01  2.17267392e+02]
 [-1.65706325e-04 -4.05743271e-04  1.00000000e+00]]
h:
[[ 4.99679031e-01  2.81181845e-01  3.40121099e+02]
 [ 2.42597923e-01  8.67774124e-01 -2.14082872e+02]
 [-4.20331898e-04  5.55467630e-04  1.00000000e+00]]
w:
[[ 6.10440031e-07  0.00000000e+00 -2.47640325e-04]
 [ 0.00000000e+00  6.10440031e-07 -2.32875528e-04]
 [-2.47640325e-04 -2.32875528e-04  1.00000000e+00]]
k:
[[1.15241364e+03 3.33058638e-10 4.05675108e+02]
 [0.00000000e+00 1.15241364e+03 3.81487970e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [13]:
# Redmi Box, vanishing points, zero skew

from itertools import combinations 
combs = list(combinations(np.arange(3), 2))

coords_redmi = [[[3144,2104],[5400,960],[3120,3168],[5240,1976]],\
                [[3120,3168],[3144,2104],[5240,1976],[5400,960]],\
                [[3120,3168],[1360,1616],[3144,2104],[1856,1072]]]

A = np.zeros((3,4))
v = np.asarray([find_vanishing_point(coords) for coords in coords_redmi])

for i,comb in enumerate(combs):
    p1,p2 = v[comb[0]],v[comb[1]]
    A[i] = np.asarray([p1[0]*p2[0]+p1[1]*p2[1],1,p1[0]+p2[0],p1[1]+p2[1]])

_,_,vh = np.linalg.svd(A)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)

w:
[[ 5.80327775e-09 -0.00000000e+00 -1.93928501e-05]
 [-0.00000000e+00  5.80327775e-09 -1.09885655e-05]
 [-1.93928501e-05 -1.09885655e-05  1.00000000e+00]]
k:
[[1.25524511e+04 2.65660158e-07 3.34170634e+03]
 [0.00000000e+00 1.25524511e+04 1.89351018e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [14]:
# Room Corner, vanishing points, zero skew

from itertools import combinations 
combs = list(combinations(np.arange(3), 2))

coords_corner = [[[2013,2090],[3707,792],[2002,4576],[3762,3861]],\
                [[2013,2090],[143,1309],[2002,4576],[99,4158]],\
                [[2002,4576],[2013,2090],[2321,5522],[2321,5710]]]

A = np.zeros((3,4))
v = np.asarray([find_vanishing_point(coords) for coords in coords_corner])

for i,comb in enumerate(combs):
    p1,p2 = v[comb[0]],v[comb[1]]
    A[i] = np.asarray([p1[0]*p2[0]+p1[1]*p2[1],1,p1[0]+p2[0],p1[1]+p2[1]])

_,_,vh = np.linalg.svd(A)
x = vh.T[:,-1]
w = np.asarray([[x[0],0,x[2]],[0,x[0],x[3]],[x[2],x[3],x[1]]])
print('w:')
print(np.divide(w,w[-1,-1]))
kkt = np.linalg.pinv(w)
kkt = np.divide(kkt,kkt[-1,-1])
k = build_k_matrix(kkt,resize_f=1)
print('k:')
print(k)

w:
[[ 7.59977943e-09 -0.00000000e+00 -1.85871731e-05]
 [-0.00000000e+00  7.59977943e-09 -4.69066364e-05]
 [-1.85871731e-05 -4.69066364e-05  1.00000000e+00]]
k:
[[9.35447408e+03 6.81161133e-08 2.44575166e+03]
 [0.00000000e+00 9.35447408e+03 6.17210497e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
